# Introduction

The largest city in Turkey that I am familiar with is İstanbul. Therefore, I thought I could look at the map and check if I can catch anything of significance. 

In [1]:
import xml.etree.cElementTree as ET
import pandas as pd
import pprint
import bz2

In [2]:
DATA_FILE = "istanbul_turkey.osm"

In [3]:
bz2_data = bz2.BZ2File(DATA_FILE+".bz2").read()
open(DATA_FILE, 'wb').write(bz2_data)

I want to first check my data to see what hierarchy it has. Below functions will extract tags and hierarchy in my dataset. It will also extract attributes and child tags for each tag residing within our XML file. 

In [4]:
def getTags(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag not in tags:
            tags[elem.tag] = 0
        tags[elem.tag] += 1
    return tags

In [5]:
def print_element_info(elem):
    print elem.tag+" sample (first encounter):"
    info = {}
    for attr in elem.attrib:
        info[attr] = elem.attrib[attr]
    print info

In [6]:
# This function will traverse attributes and childs for each tag type. It will also print each tag first time it is encountered.
def define_file(filename):
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag not in tags:
            print_element_info(elem)
            tags[elem.tag] = {"count":0,"attribs":set(),"childs":set()}
        tags[elem.tag]["count"] += 1
        for attr in elem.attrib:
            tags[elem.tag]["attribs"].add(attr)
        for elem_ in elem.getiterator():
            if elem_ != elem:
                tags[elem.tag]["childs"].add(elem_.tag)
    return tags

In [7]:
file_tag_stats = define_file(DATA_FILE)

bounds sample (first encounter):
{'minlat': '40.738', 'maxlon': '29.678', 'minlon': '28.313', 'maxlat': '41.421'}
tag sample (first encounter):
{'k': 'highway', 'v': 'motorway_junction'}
node sample (first encounter):
{'changeset': '11617084', 'uid': '683385', 'timestamp': '2012-05-16T19:43:19Z', 'lon': '29.047924', 'version': '5', 'user': 'nucularbum', 'lat': '41.0222538', 'id': '21107200'}
nd sample (first encounter):
{'ref': '482245611'}
way sample (first encounter):
{'changeset': '12637783', 'uid': '136321', 'timestamp': '2012-08-06T19:32:43Z', 'version': '14', 'user': 'Teddy73', 'id': '4341858'}
member sample (first encounter):
{'role': 'city', 'ref': '1882099475', 'type': 'node'}
relation sample (first encounter):
{'changeset': '39135345', 'uid': '2263457', 'timestamp': '2016-05-06T05:55:12Z', 'version': '33', 'user': 'tbolat', 'id': '10692'}
osm sample (first encounter):
{'timestamp': '2017-07-19T15:02:02Z', 'version': '0.6', 'generator': 'osmconvert 0.8.5'}


In [8]:
df = pd.DataFrame(file_tag_stats)
df

,bounds,member,nd,node,osm,relation,tag,way
attribs,"{minlat, maxlon, minlon, maxlat}","{role, ref, type}",{ref},"{changeset, uid, timestamp, lon, version, user...","{timestamp, version, generator}","{changeset, uid, timestamp, version, user, id}","{k, v}","{changeset, uid, timestamp, version, user, id}"
childs,{},{},{},{tag},"{node, nd, bounds, member, tag, relation, way}","{member, tag}",{},"{tag, nd}"
count,1,8034,1498568,1164528,1,694,370703,192562


From above table we can see that the main tag types are **tags**, **ways**, **nodes**, and **nds**. I am going to ignore nd types since they only provide some references. 

In [9]:
df["node"].attribs

{'changeset', 'id', 'lat', 'lon', 'timestamp', 'uid', 'user', 'version'}

Historically, Istanbul had been a very diverse city. Until recent decades, there were a large number of followers of different religions, particularly Ortodox Christians. Therefore, I would like to explore into religious places in Istanbul, and see what is current distribution of religious places in Istanbul. This data might provide insight into proportion of Istanbulites into different religious groups. 

I do not know what types of places are there which are tagged as religious, so I think I should first look into nodes/ways tagged with religion key.

Functions below will explore XML to find stats and sample entries for **element tag** - **tag key** relationships. 

In [10]:
def isTypeOf(tag_,key):
    return (key == tag_.attrib['k'])

In [11]:
def add_type(result,value):
    if value in result:
        result[value]+=1
    else:
        result[value] = 1

In [12]:
def get_elem_tags(elem):
    result = {}
    for tag_ in elem.iter("tag"):
        result[tag_.attrib["k"]] = tag_.attrib["v"]
    return result

In [13]:
def get_stats_with_tag_key(filename,tag,key):
    result = {}
    for event, elem in ET.iterparse(filename, events=("start",)):
        if elem.tag == tag:
            for tag_ in elem.iter("tag"):
                if(isTypeOf(tag_,key)):
                    add_type(result,tag_.attrib['v'])
                    if(result[tag_.attrib['v']]==1):
                        pprint.pprint(get_elem_tags(elem))
    return result

I am expecting religious places being in *node* tag type. Therefore, let's first look into nodes tagged with *religion*:

In [14]:
get_stats_with_tag_key(DATA_FILE,"node","religion")

{'historic': 'memorial',
 'name': u'Barbaros Hayrettin T\xfcrbesi',
 'religion': 'muslim'}
{'amenity': 'place_of_worship',
 'name': u'Be\u015fikta\u015f Panayia Rum Ortodoks Kilisesi Vakf\u0131',
 'religion': 'christian'}
{'amenity': 'place_of_worship',
 'name': u'Ortak\xf6y Etzahayim Musevi Sinagonu',
 'religion': 'jewish'}
{'amenity': 'place_of_worship',
 'name': u'U\xe7an Spagetti Canavar\u0131 Tap\u0131na\u011f\u0131',
 'religion': 'pastafarian'}


{'christian': 15, 'jewish': 4, 'muslim': 148, 'pastafarian': 1}

I am seeing that there are Christian, Jewish, Muslim and Pastafarian places listed. Where most of nodes belong to Muslims in an unsurprising way.

Let's also look into ways so that we don't miss anything of value. 

In [15]:
get_stats_with_tag_key(DATA_FILE,"way","religion")

{'amenity': 'place_of_worship',
 'building': 'yes',
 'name': u'\u015eehzadeba\u015f\u0131 Camii',
 'name:cs': u'Princova me\u0161ita',
 'name:de': 'Prinzenmoschee',
 'name:en': "Sehzade Camii Prince's Mosque",
 'name:ru': u'\u041c\u0435\u0447\u0435\u0442\u044c \u0428\u0435\u0445\u0437\u0430\u0434\u0435',
 'religion': 'muslim',
 'tourism': 'attraction',
 'wheelchair': 'yes',
 'wikipedia': u'tr:\u015eehzadeba\u015f\u0131 Camii'}
{'landuse': 'cemetery',
 'name': u'H\u0131ristiyan Mezarl\u0131\u011f\u0131',
 'religion': 'christian'}
{'landuse': 'cemetery',
 'name': u'A\u015fkenaz Musevi Cemaati Mezarl\u0131\u011f\u0131',
 'religion': 'jewish'}


{'christian': 95, 'jewish': 9, 'muslim': 1923}

Quite surprisingly to me, there are actually not only entries in **way-tag**, but also a lot more entries than **node-tag**. 

I think it will be plausible to merge results from both tag classes. 

# Problems Encountered

* Not all nodes have amanity tag. Therefore we will stick to last word in their names which tells about their types. 

* Most places have a varying naming issues at the end of their names. For example, Mosques have **Camii, Camisi etc**, Sinagogs have **Sinagogu, Sinangog etc.** Therefore, I will be checking last work in place names for each religion, and try to audit their types into known few types.
* There are Turkish characters in almost all names. I will be replacing this characters with their English counterpart to eliminate visual problems with character codes.

In [16]:
turkishCharMap = {
    "ç":"c",
    "Ç":"C",
    "ğ":"g",
    "Ğ":"G",
    "ı":"i",
    "İ":"I",
    "ö":"o",
    "Ö":"O",
    "ş":"s",
    "Ş":"S",
    "ü":"u",
    "Ü":"U"
}
def serializeTurkishText(text):
    text = text.encode("utf-8")
    for k,v in turkishCharMap.iteritems():
        text = text.replace(k,v)
    return text;

In [17]:
def get_last_word(name):
    return serializeTurkishText(name.split(" ")[-1]).lower()

In [18]:
def get_religious_place_types(filename,religion):
    types = {}
    place_list = {}
    for event, elem in ET.iterparse(filename, events=("start",)):
        elem_tags = get_elem_tags(elem)
        if("religion" in elem_tags and elem_tags["religion"]==religion):
            name = ""
            if "name" in elem_tags:
                name = elem_tags["name"]
            type_ = get_last_word(name)
            if type_ not in types:
                types[type_] = 1
                place_list[type_] = [elem_tags]
            else:
                types[type_] += 1
                place_list[type_].append(elem_tags)
    return (types, place_list)

In [19]:
muslim_types, muslim_place_list = get_religious_place_types(DATA_FILE,"muslim")
muslim_types

{'': 3124,
 '(insaat)': 1,
 'ada)': 6,
 'aksemsettin': 1,
 'asamasinda)': 1,
 'cami': 740,
 'camii': 267,
 'camii)': 1,
 'camii3': 1,
 'camil': 1,
 'camisi': 8,
 'cemevi': 8,
 'cmi': 1,
 'germe': 1,
 'hamami': 1,
 'hatun': 1,
 'kulliyesi': 1,
 'kursu': 1,
 'medresesi': 2,
 'mescid': 3,
 'mescidi': 19,
 'mescit': 3,
 'mezarligi': 18,
 'mezarlik': 9,
 'muftulugu': 1,
 'namazgah': 1,
 'pasa': 2,
 'serifi': 3,
 'tekkesi': 1,
 'turbesi': 3}

Many missing places(3124) are in the list. These places seem to be having no names at all. Therefore, I feel an urge to look into this entries: 

In [20]:
muslim_place_list[""]

[{'religion': 'muslim'},
 {'amenity': 'place_of_worship', 'religion': 'muslim'},
 {'religion': 'muslim'},
 {'religion': 'muslim'},
 {'amenity': 'place_of_worship',
  'name:en': 'Murat Reis Camii',
  'religion': 'muslim'},
 {'religion': 'muslim'},
 {'amenity': 'place_of_worship',
  'created_by': 'JOSM',
  'name:en': 'Murat Reia Camii',
  'religion': 'muslim'},
 {'religion': 'muslim'},
 {'religion': 'muslim'},
 {'amenity': 'place_of_worship', 'religion': 'muslim'},
 {'religion': 'muslim'},
 {'religion': 'muslim'},
 {'amenity': 'place_of_worship',
  'created_by': 'Potlatch 0.10f',
  'religion': 'muslim',
  'source': 'Yahoo'},
 {'religion': 'muslim'},
 {'amenity': 'place_of_worship',
  'created_by': 'Potlatch 0.10f',
  'religion': 'muslim'},
 {'religion': 'muslim'},
 {'amenity': 'place_of_worship',
  'created_by': 'Potlatch 0.10f',
  'religion': 'muslim'},
 {'religion': 'muslim'},
 {'religion': 'muslim'},
 {'amenity': 'place_of_worship',
  'created_by': 'Potlatch 0.10f',
  'religion': 'mus

## New Problems
* It seems some have names only in English. I will add these entries with their English names. 
* Apart from this, most of places seem to be holding no information but the religion tag itself. Since, we have no chance of making guesses about this I will simply ignore these entries. 
* Lastly, there are some entries with some variables that could help to extract their types.

In [21]:
attributes_to_check = ["amenity","name:en","building","source"]
def getProminentAttributes(list_):
    result = {}
    for elem in list_:
        for k,v in elem.iteritems():
            if k in attributes_to_check:
                if k not in result:
                    result[k] = set(v)
                else:
                    result[k].add(v)
    return result

In [22]:
pprint.pprint(getProminentAttributes(muslim_place_list[""]))

{'amenity': set(['_',
                 'a',
                 'c',
                 'e',
                 'f',
                 'grave_yard',
                 'h',
                 'i',
                 'l',
                 'o',
                 'p',
                 'place_of_worship',
                 'r',
                 's',
                 'w']),
 'building': set(['e', 'mosque', 's', 'y', 'yes']),
 'name:en': set([' ',
                 'C',
                 'M',
                 'Murat Reia Camii',
                 'R',
                 'a',
                 'e',
                 'i',
                 'm',
                 'r',
                 's',
                 't',
                 'u']),
 'source': set(['Bing',
                'Y',
                'Yahoo',
                'a',
                'bing',
                'h',
                'local_knowledge; Bing',
                'o'])}


It seems apart from "name:en" attribute none of above elements in sets provide enough info to extract type of the place. Therefore we will ignore the rest.

Now let's also look into other types that dont make much sense:

In [23]:
muslim_types

{'': 3124,
 '(insaat)': 1,
 'ada)': 6,
 'aksemsettin': 1,
 'asamasinda)': 1,
 'cami': 740,
 'camii': 267,
 'camii)': 1,
 'camii3': 1,
 'camil': 1,
 'camisi': 8,
 'cemevi': 8,
 'cmi': 1,
 'germe': 1,
 'hamami': 1,
 'hatun': 1,
 'kulliyesi': 1,
 'kursu': 1,
 'medresesi': 2,
 'mescid': 3,
 'mescidi': 19,
 'mescit': 3,
 'mezarligi': 18,
 'mezarlik': 9,
 'muftulugu': 1,
 'namazgah': 1,
 'pasa': 2,
 'serifi': 3,
 'tekkesi': 1,
 'turbesi': 3}

In [24]:
muslim_place_list["(insaat)"]

[{'amenity': 'place_of_worship',
  'building': 'yes',
  'name': u'Yeni Zeynebiye Camii (\u0130n\u015faat)',
  'religion': 'muslim'}]

This is a mosque with a note stating it's in reconstruction. Therefore, let's remove descriptions in paranthesis, and try again to see if things improve.

In [25]:
import re
def remove_paranthesis(text):
    return re.sub(r'\(.*\)', '', text).strip()

In [26]:
#Override the function to handle paranthesis:
def get_last_word(name):
    return serializeTurkishText(remove_paranthesis(name).split(" ")[-1]).lower()

In [27]:
muslim_types, muslim_place_list = get_religious_place_types(DATA_FILE,"muslim")
muslim_types

{'': 3124,
 'aksemsettin': 1,
 'cami': 741,
 'camii': 269,
 'camii3': 1,
 'camil': 1,
 'camisi': 8,
 'cemevi': 8,
 'cmi': 1,
 'germe': 1,
 'hamami': 1,
 'hatun': 1,
 'kulliyesi': 1,
 'kursu': 1,
 'medresesi': 2,
 'mescid': 3,
 'mescidi': 19,
 'mescit': 3,
 'mezarligi': 24,
 'mezarlik': 9,
 'muftulugu': 1,
 'namazgah': 1,
 'pasa': 2,
 'serifi': 3,
 'tekkesi': 1,
 'turbesi': 3}

Types listed above provide enough insight into types of places therefore we will now define mappings for types and typo fixes.

I decided to set below types:

* Mosque
* Islamic School
* Graveyard
* Other

In [28]:
mapping = {
    "muslim":{
        'aksemsettin': "Other",
        'cami': "Mosque",
        'camii': "Mosque",
        'camii3': "Mosque",
        'camil': "Mosque",
        'camisi': "Mosque",
        'cemevi': "Mosque",
        'cmi': "Mosque",
        'germe': "Other",
        'hamami': "Other",
        'hatun': "Other",
        'kulliyesi': "Islamic School",
        'kursu': "Islamic School",
        'medresesi': "Islamic School",
        'mescid': "Mosque",
        'mescidi': "Mosque",
        'mescit': "Mosque",
        'mezarligi': "Graveyard",
        'mezarlik': "Graveyard",
        'muftulugu': "Other",
        'namazgah': "Mosque",
        'pasa': "Other",
        'serifi': "Mosque",
        'tekkesi': "Islamic School",
        'turbesi': "Other"
    }
}
typo_fixes = {
    "muslim":{
        'camii': "cami",
        'camii3': "cami",
        'camil': "cami",
        'camisi': "cami",
        'cmi': "cami"
    }
}

In [29]:
christian_types, christian_place_list = get_religious_place_types(DATA_FILE,"christian")
christian_types

{'': 127,
 'ayazmasi': 3,
 'church': 4,
 'kabristani': 1,
 'katedrali': 1,
 'kilesi': 1,
 'kiliesi': 1,
 'kilise': 2,
 'kilisesi': 62,
 'manastiri': 6,
 'metropolitligi': 1,
 'mezarligi': 8,
 'mongols': 1,
 'nikola': 1,
 'patrikhanesi': 1,
 'phokas': 1,
 'vakfi': 1,
 '\xd0\xa1\xd1\x82\xd0\xb5\xd1\x84\xd0\xb0\xd0\xbd\xe2\x80\x9c': 1}

In [30]:
print '\xd0\xa1\xd1\x82\xd0\xb5\xd1\x84\xd0\xb0\xd0\xbd\xe2\x80\x9c'

Стефан“


Place types seem to be alright. But lets dive into places with no names: 

In [31]:
pprint.pprint(getProminentAttributes(christian_place_list[""]))

{'amenity': set(['_',
                 'a',
                 'c',
                 'e',
                 'f',
                 'h',
                 'i',
                 'l',
                 'o',
                 'p',
                 'place_of_worship',
                 'r',
                 's',
                 'w']),
 'building': set(['e', 's', 'y', 'yes'])}


It's seems places with no names are no more that entries with almost no info. Therefore, I will be ignoring these ones also. 

Let's also add type mapping and typo fixes mapping. Our types will be as following:

* Church
* Graveyard
* Monastery
* Other

In [32]:
mapping["christian"] = {
    'ayazmasi': "Other",
    'church': "Church",
    'kabristani': "Graveyard",
    'katedrali': "Church",
    'kilesi': "Church",
    'kiliesi': "Church",
    'kilise': "Church",
    'kilisesi': "Church",
    'manastiri': "Monastery",
    'metropolitligi': "Other",
    'mezarligi': "Graveyard",
    'mongols': "Other",
    'nikola': "Other",
    'patrikhanesi': "Other",
    'phokas': "Other",
    'vakfi': "Other",
    'Стефан“':"Other"
}
typo_fixes["christian"] = {
    'kilesi': "Kilisesi",
    'kiliesi': "Kilisesi"
}

In [33]:
jewish_types, jewish_place_list = get_religious_place_types(DATA_FILE,"jewish")
jewish_types

{'': 15,
 'mezarligi': 3,
 'neve-shalom': 1,
 'sinagog': 1,
 'sinagogu': 6,
 'sinagonu': 1}

In [34]:
pprint.pprint(getProminentAttributes(jewish_place_list[""]))

{'amenity': set(['_',
                 'a',
                 'c',
                 'e',
                 'f',
                 'h',
                 'i',
                 'l',
                 'o',
                 'p',
                 'r',
                 's',
                 'w']),
 'building': set(['e', 's', 'y'])}


There seems to be nothing to extract from places with no name, so we will ignore them.

In [35]:
jewish_place_list['neve-shalom']

[{'amenity': 'place_of_worship',
  'building': 'yes',
  'name': 'Neve-Shalom',
  'religion': 'jewish'}]

Let's add our type and typo fixes mappings. List of types is provided below:

* Synagogue
* Graveyard


In [36]:
mapping["jewish"] = {
    'mezarligi': "Graveyard",
    'neve-shalom': "Synagogue",
    'sinagog': "Synagogue",
    'sinagogu': "Synagogue",
    'sinagonu': "Synagogue"
}
typo_fixes["jewish"] = {
    'sinagonu': "Sinagogu"
}

In [37]:
pastafarian_types, pastafarian_place_list = get_religious_place_types(DATA_FILE,"pastafarian")
pastafarian_types

{'': 1, 'tapinagi': 1}

In [38]:
pastafarian_place_list[""]

[{'religion': 'pastafarian'}]

An empty place. We will ignore this.
* I will tag the only **Tapınak** as temple. 

In [39]:
mapping["pastafarian"] = {
    'tapinagi': "Temple"
}
typo_fixes["pastafarian"] = {}

# Data Extraction

Since we don't have any attribute in religious places list we will have a single SQL table.

In [40]:
import sqlite3
sqlite_file = 'istanbul_osm.sqlite'
table = "religious_places"
columns = ["id","name","lat","lon","religion","type"]

conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

**Create table, and drop if table already exists:**

In [41]:
c.execute('DROP TABLE {tn}'\
        .format(tn=table))
c.execute('CREATE TABLE {tn} ({columns})'\
        .format(tn=table, columns=",".join(columns)))

In [42]:
religions = ['christian', 'jewish', 'muslim', 'pastafarian']

In [43]:
def get_name(elem_tags):
    if "name" in elem_tags:
        return elem_tags["name"]
    elif "name:en" in elem_tags:
        return elem_tags["name:en"]
    else:
        return ""

In [44]:
def get_fixed_name(elem_tags):
    name = get_name(elem_tags)
    name_words = name.split(" ")[0:-1]
    type_identifier = get_last_word(name)
    if type_identifier in typo_fixes[elem_tags["religion"]]:
        name_words.append(typo_fixes[elem_tags["religion"]][type_identifier])
        return " ".join(name_words)
    return name

In [45]:
def get_type(elem_tags,name):
    type_identifier = get_last_word(name)
    religion = elem_tags["religion"]
    if type_identifier in mapping[religion]:
        return mapping[religion][type_identifier]
    else:
        pprint.pprint(["unknown type",elem_tags,type_identifier])
        return "Unknown" 

In [46]:
def get_religious_places(filename):
    places = []
    for event, elem in ET.iterparse(filename, events=("start",)):
        elem_tags = get_elem_tags(elem)
        if "religion" in elem_tags and elem_tags["religion"] in religions:
            name = get_fixed_name(elem_tags)
            if(len(name)>0):
                type_ = get_type(elem_tags,name)
                elem_tags["name"] = name
                elem_tags["type"] = type_
                for column in columns:
                    if column in elem.attrib:
                        elem_tags[column] = elem.attrib[column]
                places.append(elem_tags)
    return places

**Below code will extract all places for all religions:**

In [47]:
religious_places = get_religious_places(DATA_FILE)

In [48]:
df = pd.DataFrame(religious_places,columns=columns)
df

,id,name,lat,lon,religion,type
0,269497288,Barbaros Hayrettin Türbesi,41.0419227,29.0068359,muslim,Other
1,269706604,Ertuğrul Tekke cami,41.0456489,29.0085216,muslim,Mosque
2,269707397,Beşiktaş Panayia Rum Ortodoks Kilisesi Vakfı,41.0436679,29.0050595,christian,Other
3,278092559,Murat Reis cami,41.0184464,29.0275916,muslim,Mosque
4,278102132,Murat Reia cami,41.0231166,29.0237092,muslim,Mosque
5,278471143,Karacaahmet Cemevi,41.0132479,29.0201048,muslim,Mosque
6,290538729,Isman Reis cami,41.1145299,29.0621858,muslim,Mosque
7,351639244,Burgazada cami,40.8829893,29.0684726,muslim,Mosque
8,354969292,Hacı Nimet Özden cami,41.0681321,29.0101802,muslim,Mosque
9,355123822,Germe,41.2738386,28.6616036,muslim,Other


**Let's check if any of places comes with missing fields:**

In [49]:
def check_data_integrity(places):
    missing = []
    for place in places:
        for column in columns:
            if column not in place:
                missing.append(place)
                break
    return missing

In [50]:
missing = check_data_integrity(religious_places)

In [51]:
print str(len(missing))+" out of "+str(len(religious_places))+" places will have NA values in database."

1122 out of 1217 places will have NA values in database.


**Function below will process all results and insert them into SQL table.**

In [52]:
def insert_values(cursor,table,columns,values):
    sql_values = ""
    for value in values:
        values_str = ""
        for column in columns:
            if len(values_str)>0:
                values_str += ","
            if column in value:
                #replace quotes with double quotes in texts: special chars in SQL
                values_str += "'"+value[column].replace("'","''")+"'"
            else:
                values_str += "NULL"
        if len(sql_values)>0:
            sql_values += ","
        sql_values += "("+values_str+")"
    c.execute('INSERT INTO {tn} ({columns}) values {values}'
              .format(tn=table, columns=",".join(columns), values=sql_values.encode("utf-8")))

In [53]:
insert_values(c,table,columns,religious_places)
conn.commit()

# Sample SQL Queries

### Top Religios Place Types

In [54]:
c.execute("select type, count(type) as count from religious_places group by type order by count desc")
pd.DataFrame(c.fetchall(),columns=["Type","Count"])

,Type,Count
0,Mosque,1060
1,Church,71
2,Graveyard,45
3,Other,20
4,Synagogue,9
5,Monastery,6
6,Islamic School,5
7,Temple,1


### Top Religious Place Names

In [55]:
c.execute("select name, count(type) as count from religious_places group by name order by count desc limit 10 offset 0")
pd.DataFrame(c.fetchall(),columns=["Type","Count"])

,Type,Count
0,Cami,219
1,Mezarlık,8
2,Mevlana Cami,6
3,Fatih cami,4
4,Yunus Emre Cami,4
5,Akşemsettin Cami,3
6,Berat Cami,3
7,Huzur Cami,3
8,Hz. Ali Cami,3
9,Hz. Ebubekir cami,3


### Religion Rank by # of Places

In [56]:
c.execute("select religion, count(type) as count from religious_places group by religion order by count desc")
pd.DataFrame(c.fetchall(),columns=["Type","Count"])

,Type,Count
0,muslim,1108
1,christian,96
2,jewish,12
3,pastafarian,1
